# Lending Club loan data

El presente proyecto tiene como objetivo probar diferentes modelos de aprendizaje automático para determinar si un cliente tendrá problemas con el pago de su deuda o no,  de acuerdo a diversas características como el riesgo o incumplimiento por parte del prestatario a lo largo de los años.

https://www.kaggle.com/wordsforthewise/lending-club

In [50]:
import io
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import clone
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
%matplotlib inline
pd.options.display.max_rows = 1000

In [39]:
%ls

data_acc_200k.csv  sample_data/


# **Procesamiento de la data inicial**

In [5]:
loans = pd.read_csv('data_acc_200k.csv', low_memory=False)
loans.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,...,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,39651438,NaN,32000.0,32000.0,32000.0,60 months,10.49,687.65,B,B3,Public Service,10+ years,MORTGAGE,120000.0,Verified,Feb-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,919xx,CA,24.05,0.0,Oct-1981,735.0,739.0,0.0,NaN,NaN,20.0,0.0,39687.0,57.8,42.0,w,7181.91,...,556496.0,103647.0,64100.0,72197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,16411620,NaN,9600.0,9600.0,9600.0,36 months,12.99,323.42,C,C1,NaN,NaN,RENT,21900.0,Verified,May-2014,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,331xx,FL,10.03,0.0,Apr-2001,685.0,689.0,1.0,NaN,118.0,13.0,1.0,4509.0,38.9,20.0,w,0.00,...,11600.0,4509.0,2400.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,2,45122316,NaN,4000.0,4000.0,4000.0,36 months,6.68,122.93,A,A3,System Analyst,4 years,MORTGAGE,83000.0,Not Verified,Apr-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,major_purchase,Major purchase,333xx,FL,19.53,0.0,Sep-2003,780.0,784.0,0.0,NaN,NaN,16.0,0.0,1564.0,17.2,25.0,w,0.00,...,222616.0,64253.0,5600.0,76154.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,3,125356772,NaN,6025.0,6025.0,6025.0,36 months,10.91,197.00,B,B4,Admin assistant,10+ years,RENT,52000.0,Not Verified,Dec-2017,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,021xx,MA,9.16,1.0,Jun-2005,740.0,744.0,1.0,15.0,NaN,11.0,0.0,2706.0,12.8,25.0,w,0.00,...,32227.0,5559.0,11000.0,11127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,4,128490686,NaN,25000.0,25000.0,25000.0,60 months,26.30,752.96,E,E5,Coordinator,10+ years,OWN,65000.0,Verified,Feb-2018,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,926xx,CA,36.26,0.0,Jul-1999,715.0,719.0,1.0,37.0,NaN,19.0,0.0,49461.0,24.7,33.0,w,21954.14,...,257219.0,97647.0,179400.0,65719.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data=loans

In [7]:
loans.shape

(22445, 152)

# **Preprocesamiento de los datos**

In [8]:
a = ((data.isnull().sum())/len(data)*100).sort_values()

In [9]:
data2 = data.drop(data.loc[:, data.isna().mean().sort_values() > 0.3].columns, axis=1)

In [10]:
#Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
threshold = 0.66

unique_percentages = data2.nunique() / len(data2)

# en caso se una variable numérica continua, es probable que todos los valores sean únicos (?)

criteria = unique_percentages > threshold

columns_to_filter = unique_percentages[criteria].keys()

In [11]:
#Eliminamos las variables seleccionadas en el paso anterior
data2.drop(columns_to_filter, axis=1, inplace=True)

In [12]:
#Determinamos que variables son del tipo númerico y cuales son categóricas
numeric_columns = list()
categorical_columns = list()
dictionary_of_columns_with_index_to_categorical = dict()
dictionary_of_columns_with_categorical_to_index = dict()

for column in data2:
#Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(data2[column]):
        numeric_columns.append(column)
    else:
        #Modificamos el tipo de dato de la variable mediante "astype"
        data2[column] = data2[column].astype("category")

        #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
        if not pd.api.types.is_categorical_dtype(data2[column]):
            raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

        dictionary_of_columns_with_index_to_categorical[column] = dict()
        dictionary_of_columns_with_categorical_to_index[column] = dict()

        #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
        for index, category in enumerate(data2[column].cat.categories):
            dictionary_of_columns_with_index_to_categorical[column][index] = category
            dictionary_of_columns_with_categorical_to_index[column][category] = index

        categorical_columns.append(column)

In [13]:
#Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
median_of_numeric_columns = data2[numeric_columns].median()

data2[numeric_columns] = data2[numeric_columns].fillna(median_of_numeric_columns)

In [14]:
data2 = data2.drop('title', axis=1)

In [15]:
data2 = data2.drop('emp_title',axis=1)

In [16]:
data2 = data2.drop('emp_length', axis=1)

In [17]:
data2 = data2.dropna()

In [18]:
data.shape

(22445, 152)

In [19]:
data2 = data2.drop('grade', axis=1)

In [20]:
data2.shape

(22415, 78)

In [21]:
dat= (data2.nunique()/data2.shape[0]>0.5) == True
data2 = data2.drop('last_pymnt_amnt', axis=1)

KeyError: ignored

In [22]:
#Juntar las clases de loan_status
#“0”: "Fully Paid", "Current", “In Grace Period” y “Does not meet the credit policy. Status:Fully Paid”
#“1”: "Charged Off", “Late (31-120 days)”, “Late (16-30 days)” y “Does not meet the credit policy. Status:Charged Off”
#Los registros con el valor de “Default” serán descartados.

In [23]:
data2 = data2.drop(data2.loc[data2['loan_status']=="Default"].index, axis=0)

In [24]:
data2['target']=1

In [25]:
data2.loc[(data2['loan_status']=="Fully Paid"), 'target'] = 0
data2.loc[(data2['loan_status']=="Current"), 'target'] = 0
data2.loc[(data2['loan_status']=="In Grace Period"), 'target'] = 0
data2.loc[(data2['loan_status']=="Does not meet the credit policy. Status:Fully Paid"), 'target'] = 0

In [26]:
data2 = data2.drop('loan_status', axis=1)

In [27]:
data2.dtypes
lista=list()
for i in data2.columns:
  if  (data2[i].dtypes!=float) & (data2[i].dtypes!=int):
    lista.append(i)
lista
#categoricos todos los que contegan letras, true y false tambien serían categoricos
data3 = pd.get_dummies(data2, columns=lista)

In [28]:
data3['target'].value_counts()

0    19447
1     2968
Name: target, dtype: int64

In [29]:
data3 = data3.reset_index(drop=True)

## **Conjunto de entrenamiento, validación y test**

sklearn.model_selection.train_test_split separa los índices del conjunto de datos en un 90% para entrenamiento y validación (indices_trainval), y 10% para pruebas (indices_test). Luego separa indices_trainval en un 85% para entrenamiento (indices_train), y 15% para validación (indices_val).

In [30]:
from sklearn.model_selection import train_test_split

indices = data3.index.values

indices_trainval, indices_test = train_test_split(indices, test_size=0.10, random_state=0)
indices_train, indices_val =train_test_split(indices_trainval, test_size=0.15, random_state=0)

# El conjunto de entrenamiento es: data3.loc[indices_train]
# El conjunto de validación es: data3.loc[indices_val]
# El conjunto de prueba es: data3.loc[indices_test] 

print('Los primeros 5 índices del conjunto de entrenamiento son: ', indices_train[:5])

Los primeros 5 índices del conjunto de entrenamiento son:  [ 2216 14686 11264  9476 19311]


Empleando sklearn.preprocessing.StandardScaler se ha primero normalizado los datos (usando el conjunto de entrenamiento), y luego se ha separado los valores de X_train, X_val, X_trainval, X_test y sus correspondientes Y_train, Y_val, Y_trainval, Y_test.

In [31]:
from sklearn.preprocessing import StandardScaler

feature_columns = data3.columns.drop('target')

X = data3[feature_columns].values.astype(float)
Y = data3['target'].values.astype(int)

scaler = StandardScaler().fit(X[indices_train])

In [32]:
X_scaled = scaler.transform(X)

X_train = X_scaled[indices_train]
X_val = X_scaled[indices_val]
X_trainval = X_scaled[indices_trainval]
X_test = X_scaled[indices_test]

Y_train = Y[indices_train]
Y_val = Y[indices_val]
Y_trainval = Y[indices_trainval]
Y_test = Y[indices_test]

Otra forma

In [ ]:
X_train = scaler.transform(X[indices_train])
X_val = scaler.transform(X[indices_val])
X_trainval = scaler.transform(X[indices_trainval])
X_test = scaler.transform(X[indices_test])

Y_train = scaler.transform(Y[indices_train])
Y_val = scaler.transform(Y[indices_val])
Y_trainval = scaler.transform(Y[indices_trainval])
Y_test = scaler.transform(Y[indices_test])

In [33]:
X_train.shape

(17147, 1964)

In [34]:
X_val.shape

(3026, 1964)

# **Implementación de los modelos**

# Medidas de calidad

In [35]:
from sklearn import metrics

def print_binary_confusion_matrix(y_true, y_pred):
  confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
  TN = confusion_matrix[0,0]
  FN = confusion_matrix[1,0]
  FP = confusion_matrix[0,1]
  TP = confusion_matrix[1,1]
  exactitud = (TP+TN)/(TP+TN+FP+FN)
  precision = TP/(TP+FP)
  exhaustividad = TP/(TP+FN)
  f1 = 2*(precision*exhaustividad)/(precision + exhaustividad)
  print('Accuracy : {:.3f}'.format(exactitud))
  print('Precision: {:.3f}'.format(precision))
  print('Recall   : {:.3f}'.format(exhaustividad))  
  print('F1       : {:.3f}'.format(f1))
  print()

# Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

In [36]:
########################Cross validation###############################
'''Cs = np.logspace(-4,-1,19)
print('Se probará regresión logística con los siguientes valores de C: ')
print(Cs)
print()
mejor_modelo = None
mejor_score = 0
C=0
for i in Cs:
    modelo_lr = LogisticRegression(penalty='l2',C=i)
######entrenamiento con X_train##############
    modelo_lr.fit(X_train,Y_train)
    valor=modelo_lr.score(X_val,Y_val)
    if valor > mejor_score :
      mejor_score=modelo_lr.score(X_val,Y_val)
      mejor_modelo = modelo_lr
      C=modelo_lr.get_params()['C']

#modelo_lr=LogisticRegression(penalty='l2',C=C).fit(X_train,Y_train)
print('Mejor valor de C encontrado:')
print(modelo_lr.get_params()['C'])'''

Se probará regresión logística con los siguientes valores de C: 
[0.0001     0.00014678 0.00021544 0.00031623 0.00046416 0.00068129
 0.001      0.0014678  0.00215443 0.00316228 0.00464159 0.00681292
 0.01       0.01467799 0.02154435 0.03162278 0.04641589 0.06812921
 0.1       ]

Mejor valor de C encontrado:
0.1


In [40]:
#####################Parámetros del modelo##########################
Cs = np.logspace(-4,-1,19)
grid = {'C': Cs}

print('Los valores a probar en la búsqueda por grilla son:')
print(grid)
print()
########################Grid search###############################
model = LogisticRegression(penalty='l2')
modelo = GridSearchCV(estimator = model, 
                        param_grid = grid, 
                        cv = 3,          # Validación cruzada 3-fold
                        verbose=2, 
                        n_jobs = -1 )     # Paralelizar en todos los cores disponibles
modelo.fit(X_train, Y_train)
mejor_modelo_LR = modelo.best_estimator_
print()
print('Los hiperparámetros del mejor modelo son:')
print(modelo.best_params_)
print()
print('Exactitud luego de búsqueda en grilla en entrenamiento:', mejor_modelo_LR.score(X_train, Y_train))
print('Exactitud luego de búsqueda en grilla en validación:', mejor_modelo_LR.score(X_val, Y_val))
print('Exactitud luego de búsqueda en grilla en prueba:', mejor_modelo_LR.score(X_test,Y_test))

Los valores a probar en la búsqueda por grilla son:
{'C': array([0.0001    , 0.00014678, 0.00021544, 0.00031623, 0.00046416,
       0.00068129, 0.001     , 0.0014678 , 0.00215443, 0.00316228,
       0.00464159, 0.00681292, 0.01      , 0.01467799, 0.02154435,
       0.03162278, 0.04641589, 0.06812921, 0.1       ])}
Fitting 3 folds for each of 19 candidates, totalling 57 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:  1.0min finished



Los hiperparámetros del mejor modelo son:
{'C': 0.046415888336127774}

Exactitud luego de búsqueda en grilla en entrenamiento: 0.9940514375692541
Exactitud luego de búsqueda en grilla en validación: 0.9662921348314607
Exactitud luego de búsqueda en grilla en prueba: 0.9701159678858162


In [41]:
# Volvemos a ajustar con el conjunto de entrenamiento + validación (entrenamiento con toda la data)
mejor_modelo_LR.fit(X_trainval, Y_trainval)
print('Conjunto de entrenamiento + validación:')
print_binary_confusion_matrix(Y_trainval, mejor_modelo_LR.predict(X_trainval))
print('Conjunto de prueba :')
print_binary_confusion_matrix(Y_test, mejor_modelo_LR.predict(X_test))

Conjunto de entrenamiento + validación:
Accuracy : 0.994
Precision: 0.988
Recall   : 0.963
F1       : 0.975

Conjunto de prueba :
Accuracy : 0.973
Precision: 0.923
Recall   : 0.875
F1       : 0.898



# Support Vector Machines

In [ ]:
from sklearn.svm import SVC

In [ ]:
########################Cross validation###############################
'''Cs = np.logspace(-2,2,9)  # ~ [0.01, 0.03, 0.1, ..., 100]
gammas = np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
#aplicar el cross-validation
#encontrar los mejores parametros para C y gamma
mejor_modelo = None
mejor_score = 0
for C in Cs:
    for gamma in gammas:        
        model = SVC(kernel='rbf', C=C, gamma=gamma)
        #entrenamiento con X_train
        model.fit(X_train, Y_train)        
        score_val = model.score(X_val, Y_val)        
        if score_val > mejor_score:
            mejor_score = score_val
            mejor_modelo = model

#para hacerlo más rapido usar GridSearchCV ahí se obtienen los mejores parámetros
print ('Mejor valor de C: %0.4f' % mejor_modelo.get_params()['C'])
print ('Mejor valor de gamma: %0.4f' % mejor_modelo.get_params()['gamma'])
print ('Accuracy en el conjunto de entrenamiento: %0.4f' % mejor_modelo.score(X_train, Y_train))
print ('Accuracy en el conjunto de validación   : %0.4f' % mejor_modelo.score(X_val, Y_val))'''

In [ ]:
#####################Parámetros del modelo##########################
Cs = np.logspace(-2,2,9)  # ~ [0.01, 0.03, 0.1, ..., 100]
gammas = np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
grid = {'C': Cs,'gamma': gammas}

print('Los valores a probar en la búsqueda por grilla son:')
print(grid)
########################Grid search###############################
model = SVC(kernel='rbf')
modelo = GridSearchCV(estimator = model, 
                        param_grid = grid, 
                        cv = 3,          # Validación cruzada 3-fold
                        verbose=2, 
                        n_jobs = -1      # Paralelizar en todos los cores disponibles
                        )
modelo.fit(X_train, Y_train)
mejor_modelo_SVM = modelo.best_estimator_
print()
print('Los hiperparámetros del mejor modelo son:')
print(modelo.best_params_)
print()
print('Exactitud luego de búsqueda en grilla en entrenamiento:', mejor_modelo_SVM.score(X_train, Y_train))
print('Exactitud luego de búsqueda en grilla en validación:', mejor_modelo_SVM.score(X_val, Y_val))
print('Exactitud luego de búsqueda en grilla en prueba:', mejor_modelo_SVM.score(X_test,Y_test))

In [ ]:
# Volvemos a ajustar con el conjunto de entrenamiento + validación (entrenamiento con toda la data)
mejor_modelo_SVM.fit(X_trainval, Y_trainval)
print('Conjunto de entrenamiento + validación:')
print_binary_confusion_matrix(Y_trainval, mejor_modelo_SVM.predict(X_trainval))
print('Conjunto de prueba :')
print_binary_confusion_matrix(Y_test, mejor_modelo_SVM.predict(X_test))

# Redes Neuronales

# Gradient Boosting

In [46]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#####################Parámetros del modelo##########################
learning_rate  = [0.01,0.1,0.5,1]
n_estimators = [200,250,300,400]
max_features = [9, 10, 11]
max_depth = [2,5,10]
min_samples_leaf = [4]

grid = {'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf}

print('Los valores a probar en la búsqueda por grilla son:')
print(grid)
########################Grid search###############################
gradBoo = GradientBoostingClassifier(random_state=0)
modelo = GridSearchCV(estimator = gradBoo, 
                        param_grid = grid, 
                        cv = 3,          # Validación cruzada 3-fold
                        verbose=2, 
                        n_jobs = -1      # Paralelizar en todos los cores disponibles
                        )
modelo.fit(X_train, Y_train)
mejor_modelo_GB = modelo.best_estimator_
print()
print('Los hiperparámetros del mejor modelo son:')
print(modelo.best_params_)
print()
print('Exactitud luego de búsqueda en grilla en entrenamiento:', mejor_modelo_GB.score(X_train, Y_train))
print('Exactitud luego de búsqueda en grilla en validación:', mejor_modelo_GB.score(X_val, Y_val))
print('Exactitud luego de búsqueda en grilla en prueba:', mejor_modelo_GB.score(X_test,Y_test))
#scores_train = cross_val_score(mejor_modelo_GB, X_train, y_train, cv=5)
#scores_val = cross_val_score(mejor_modelo_GB, X_val, y_val, cv=5)
#scores_train.mean(), scores_val.mean()

In [ ]:
# Volvemos a ajustar con el conjunto de entrenamiento + validación (entrenamiento con toda la data)
mejor_modelo_GB.fit(X_trainval, Y_trainval)
print('Conjunto de entrenamiento + validación:')
print_binary_confusion_matrix(Y_trainval, mejor_modelo_GB.predict(X_trainval))
print('Conjunto de prueba :')
print_binary_confusion_matrix(Y_test, mejor_modelo_GB.predict(X_test))

# Bagging

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#####################Parámetros del modelo##########################
max_features = [9, 10, 11]
max_depth = [57, 60, 63]
min_samples_leaf = [4]
n_estimators = [1200]

grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf}

print('Los valores a probar en la búsqueda por grilla son:')
print(grid)
########################Grid search###############################
rf = RandomForestClassifier(oob_score = True)
modelo = GridSearchCV(estimator = rf, 
                        param_grid = grid, 
                        cv = 3,          # Validación cruzada 3-fold
                        verbose=2, 
                        n_jobs = -1      # Paralelizar en todos los cores disponibles
                        )
modelo.fit(X_train, Y_train)
mejor_modelo_B = modelo.best_estimator_
print()
print('Los hiperparámetros del mejor modelo son:')
print(modelo.best_params_)
print()
print('Exactitud luego de búsqueda en grilla en entrenamiento:', mejor_modelo_B.score(X_train, Y_train))
print('Exactitud luego de búsqueda en grilla en validación:', mejor_modelo_B.score(X_val, Y_val))
print('Exactitud luego de búsqueda en grilla en prueba:', mejor_modelo_B.score(X_test,Y_test))

In [ ]:
# Volvemos a ajustar con el conjunto de entrenamiento + validación (entrenamiento con toda la data)
mejor_modelo_B.fit(X_trainval, Y_trainval)
print('Conjunto de entrenamiento + validación:')
print_binary_confusion_matrix(Y_trainval, mejor_modelo_B.predict(X_trainval))
print('Conjunto de prueba :')
print_binary_confusion_matrix(Y_test, mejor_modelo_B.predict(X_test))

# Apilamiento

In [ ]:
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier

In [ ]:
#####################Selección de modelo##########################
clf1 = mejor_modelo_SVM
clf2 = mejor_modelo_GB
clf3 = mejor_modelo_B
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')
for clf, label in zip([clf1, clf2, clf3, sclf], ['XGBoost','Gradient Boosting', 'AdaBoost','StackingClassifier']):
    scores = model_selection.cross_val_score(clf, X_val, Y_val, cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


sclf.fit(X_train, Y_train)
mejor_modelo = sclf.best_estimator_
print()
print('Los hiperparámetros del mejor modelo son:')
print(modelo.best_params_)
print()
print('Exactitud luego de búsqueda en grilla en entrenamiento:', mejor_modelo.score(X_train, Y_train))
print('Exactitud luego de búsqueda en grilla en validación:', mejor_modelo.score(X_val, Y_val))
print('Exactitud luego de búsqueda en grilla en prueba:', mejor_modelo.score(X_test,Y_test))

#scores_train = cross_val_score(sclf, X_train, y_train, cv=5)
#scores_val = cross_val_score(sclf, X_val, y_val, cv=5)
#scores_train.mean(), scores_val.mean()

In [ ]:
# Volvemos a ajustar con el conjunto de entrenamiento + validación (entrenamiento con toda la data)
mejor_modelo.fit(X_trainval, Y_trainval)
print('Conjunto de entrenamiento + validación:')
print_binary_confusion_matrix(Y_trainval, mejor_modelo.predict(X_trainval))
print('Conjunto de prueba :')
print_binary_confusion_matrix(Y_test, mejor_modelo.predict(X_test))